# BERT모델 학습
 - 대화 인격과 주제 파악 분류기

구글드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/dataset
!pwd

Mounted at /content/drive/
/content/drive/My Drive/NF


In [ ]:
pip install transformers

     |████████████████████████████████| 4.2 MB 5.2 MB/s 
     |████████████████████████████████| 84 kB 2.0 MB/s 
     |████████████████████████████████| 6.6 MB 29.6 MB/s 
     |████████████████████████████████| 596 kB 51.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import gc
from transformers import BertTokenizerFast, TFBertModel

## 데이터 로딩

In [ ]:
df = pd.read_csv('prep_data.csv')
df.tail()

In [ ]:
targetcols= df.columns[:6]

### 타겟 라벨링

In [ ]:
encoders = {}
for i, col in enumerate(targetcols):
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le

df.head()

In [ ]:
df.shape

(128447, 8)

In [ ]:
le.inverse_transform([1,3,6])

array(['감사하는', '고립된', '기쁨'], dtype=object)

In [ ]:
le.classes_

array(['가난한, 불우한', '감사하는', '걱정스러운', '고립된', '괴로워하는', '구역질 나는', '기쁨', '낙담한',
       '남의 시선을 의식하는', '노여워하는', '눈물이 나는', '느긋', '당혹스러운', '당황', '두려운',
       '마비된', '만족스러운', '방어적인', '배신당한', '버려진', '부끄러운', '분노', '불안', '비통한',
       '상처', '성가신', '스트레스 받는', '슬픔', '신뢰하는', '신이 난', '실망한', '악의적인',
       '안달하는', '안도', '억울한', '열등감', '염세적인', '외로운', '우울한', '자신하는', '조심스러운',
       '좌절한', '죄책감의', '질투하는', '짜증내는', '초조한', '충격 받은', '취약한', '툴툴대는',
       '편안한', '한심한', '혐오스러운', '혼란스러운', '환멸을 느끼는', '회의적인', '후회되는', '흥분',
       '희생된'], dtype=object)

In [ ]:
decoders = {}
for col in encoders:
    print(f"====== col 변수의 라벨 =========")
    decoders[col] = {i:lbs for i, lbs in enumerate(encoders[col].classes_)}
    print(decoders[col])

In [ ]:
# import pickle
# with open('decorders.pkl', 'wb') as f:
#     pickle.dump(decoders, f)

### 토큰 변경
[Q], [A] 토큰을 사용하려면, embedding 차원이 변경되어, 사전학습 모델 사용시 성능이 떨어질 우려가 있음

따라서, 기존 [SEP] 토큰으로 변경함
- 토큰을 아예 삭제하는 것도 검토해볼 필요 있음

In [ ]:
df['Q'].iloc[-1]

'요즘 자식들과 아내가 내게 요구하는 게 점점 많아지는 거 같아서 짜증 나. [A] 자식들과 아내 때문에 마음이 편치 않으시군요. [Q] 전에는 그러지 않았는데 요즘은 점점 돈이나 게임기나 옷이나 나한테 요구하기만 해. [A] 이 상황을 극복하려면 어떻게 해야 할까요? [Q] 가족들과 한번 진중한 대화를 나눠봐야겠어. [A] 진중한 대화를 나눠보면 어떤 점이 달라질 수 있을까요? [Q] 최소한 내가 스트레스 받고 있다는 건 알릴 수 있을 것 같네.'

In [ ]:
type(df['Q'].str)

pandas.core.strings.accessor.StringMethods

In [ ]:
df['Q'].iloc[-1].replace(r'[A]', r'[SEP]')

'요즘 자식들과 아내가 내게 요구하는 게 점점 많아지는 거 같아서 짜증 나. [SEP] 자식들과 아내 때문에 마음이 편치 않으시군요. [Q] 전에는 그러지 않았는데 요즘은 점점 돈이나 게임기나 옷이나 나한테 요구하기만 해. [SEP] 이 상황을 극복하려면 어떻게 해야 할까요? [Q] 가족들과 한번 진중한 대화를 나눠봐야겠어. [SEP] 진중한 대화를 나눠보면 어떤 점이 달라질 수 있을까요? [Q] 최소한 내가 스트레스 받고 있다는 건 알릴 수 있을 것 같네.'

In [ ]:
# df['Q'] = df['Q'].str.replace(r'\[[AQ]\]', r'[SEP]')
df['Q'] = df['Q'].str.replace(r'\[A\]', r'[SEP]').str.replace(r'\[Q\]', r'[SEP]')
df.tail()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,Q,A
128442,3,0,10,2,2,17,쉬는 시간에 선생님 몰래 같은 반 애들이 친구를 때리는 걸 봐서 화가 났어. [SE...,담임 선생님께 말씀 드리면 가해 학생들이 징계를 받게 될 수 있군요.
128443,3,1,11,2,1,52,내가 좋아하는 일을 해야 할지 부모님이 원하는 일을 해야 할지 알 수 없어서 혼란스...,이 일에 대한 나의 열정을 말씀 드리려 하는군요.
128444,0,1,3,0,1,3,친구와 함께 가기로 한 여행을 나 혼자 가게 되었어. 조금 당황스러워. [SEP] ...,오랜만에 고향에 가셔서 마음껏 즐기고 오셨으면 좋겠어요.
128445,3,0,10,2,5,38,친구들과 싸움이 없는 날이 하루도 없네. [SEP] 계속 싸운다니 마음이 암담하시겠...,마음을 잘 다스려서 친구들과 잘 지냈으면 좋겠어요.
128446,1,0,4,2,2,48,요즘 자식들과 아내가 내게 요구하는 게 점점 많아지는 거 같아서 짜증 나. [SEP...,가족들과 진중한 대화를 나누고 관계가 나아지기를 바라요.


In [ ]:
df['Q'].iloc[-1]

'요즘 자식들과 아내가 내게 요구하는 게 점점 많아지는 거 같아서 짜증 나. [SEP] 자식들과 아내 때문에 마음이 편치 않으시군요. [SEP] 전에는 그러지 않았는데 요즘은 점점 돈이나 게임기나 옷이나 나한테 요구하기만 해. [SEP] 이 상황을 극복하려면 어떻게 해야 할까요? [SEP] 가족들과 한번 진중한 대화를 나눠봐야겠어. [SEP] 진중한 대화를 나눠보면 어떤 점이 달라질 수 있을까요? [SEP] 최소한 내가 스트레스 받고 있다는 건 알릴 수 있을 것 같네.'

### 학습 데이터 추가
Q열과 Q+A 열을 모두 학습 데이터로 사용 => 데이터 2배 증가

In [ ]:
df['QnA'] = df['Q'] + ' [SEP] ' + df['A']
df.tail()

,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,Q,A,QnA
128442,3,0,10,2,2,17,쉬는 시간에 선생님 몰래 같은 반 애들이 친구를 때리는 걸 봐서 화가 났어. [SE...,담임 선생님께 말씀 드리면 가해 학생들이 징계를 받게 될 수 있군요.,쉬는 시간에 선생님 몰래 같은 반 애들이 친구를 때리는 걸 봐서 화가 났어. [SE...
128443,3,1,11,2,1,52,내가 좋아하는 일을 해야 할지 부모님이 원하는 일을 해야 할지 알 수 없어서 혼란스...,이 일에 대한 나의 열정을 말씀 드리려 하는군요.,내가 좋아하는 일을 해야 할지 부모님이 원하는 일을 해야 할지 알 수 없어서 혼란스...
128444,0,1,3,0,1,3,친구와 함께 가기로 한 여행을 나 혼자 가게 되었어. 조금 당황스러워. [SEP] ...,오랜만에 고향에 가셔서 마음껏 즐기고 오셨으면 좋겠어요.,친구와 함께 가기로 한 여행을 나 혼자 가게 되었어. 조금 당황스러워. [SEP] ...
128445,3,0,10,2,5,38,친구들과 싸움이 없는 날이 하루도 없네. [SEP] 계속 싸운다니 마음이 암담하시겠...,마음을 잘 다스려서 친구들과 잘 지냈으면 좋겠어요.,친구들과 싸움이 없는 날이 하루도 없네. [SEP] 계속 싸운다니 마음이 암담하시겠...
128446,1,0,4,2,2,48,요즘 자식들과 아내가 내게 요구하는 게 점점 많아지는 거 같아서 짜증 나. [SEP...,가족들과 진중한 대화를 나누고 관계가 나아지기를 바라요.,요즘 자식들과 아내가 내게 요구하는 게 점점 많아지는 거 같아서 짜증 나. [SEP...


In [ ]:
train_all = pd.concat([df.iloc[:, :-2], df.drop(['Q', 'A'], axis=1).rename({'QnA':'Q'}, axis=1)], ignore_index=True)
train_all.tail()

,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,Q
256889,3,0,10,2,2,17,쉬는 시간에 선생님 몰래 같은 반 애들이 친구를 때리는 걸 봐서 화가 났어. [SE...
256890,3,1,11,2,1,52,내가 좋아하는 일을 해야 할지 부모님이 원하는 일을 해야 할지 알 수 없어서 혼란스...
256891,0,1,3,0,1,3,친구와 함께 가기로 한 여행을 나 혼자 가게 되었어. 조금 당황스러워. [SEP] ...
256892,3,0,10,2,5,38,친구들과 싸움이 없는 날이 하루도 없네. [SEP] 계속 싸운다니 마음이 암담하시겠...
256893,1,0,4,2,2,48,요즘 자식들과 아내가 내게 요구하는 게 점점 많아지는 거 같아서 짜증 나. [SEP...


In [ ]:
del df
gc.collect()

308

In [ ]:
# 랜덤하게 데이터 섞기
train_all = train_all.sample(frac=1, random_state=34).reset_index(drop=True)
train_all.head(10)
# train_all.to_csv('clf_train_all.csv', index=False)

,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,Q
0,0,1,6,1,2,21,돈이 없는데 내가 이렇게 병이 들 줄은 몰랐어. 이런 현실에 화가 나. [SEP] ...
1,1,1,4,2,2,9,아들이 대학을 졸업해도 취직을 안 하고 집에만 뒹굴거리고 있으니 꼴 보기 싫어. [...
2,1,0,7,2,3,26,언제부턴가 아내가 노후 자금을 모으라고 독촉을 심하게 하기 시작해서 지긋지긋해. [...
3,3,1,10,2,1,50,왜 학교 폭력 가해자는 피해자에게 사과하지 않는 걸까? 그런 걸 보면 가해자의 태도...
4,1,1,4,2,1,20,아이에게 성교육을 시키려는데 내가 먼저 부끄러운 마음이 생겨서 어색해. [SEP] ...
5,0,0,3,1,5,55,내가 담배를 피운 것 때문에 큰아들에게 폐를 이식받게 되어서 정말 미안하네. [SE...
6,3,1,10,2,3,2,동생이 학교폭력에 시달리고 있어.
7,0,1,2,1,5,27,암에 걸려서 우리 딸의 결혼식도 못 보고 죽을 것 같은데 너무 슬퍼. [SEP] 딸...
8,2,1,9,2,1,51,항상 자신의 뜻대로만 직원들이 움직여주길 바라는 상사가 끔찍하게 싫어. [SEP] ...
9,3,1,11,2,1,50,이번 시험도 망치고 난 왜 이렇게 공부를 못 할까? 한심해. [SEP] 시험 결과가...


## 정수화 및 패딩

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("klue/bert-base", truncation_side='left')
tokenizer.all_special_tokens

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [ ]:
Xs = tokenizer(train_all['Q'].tolist(), truncation=True, padding=True, max_length=512)

In [ ]:
# 메모리부족 문제를 해결하기 위해 빠르게 numpy array로 변환합니다.
# keys =list( Xs.keys())
keys = ['input_ids', 'attention_mask', 'token_type_ids']
for k in keys:
    Xs[k] = np.array(Xs[k])
    # np.savez_compressed(f'Xs_{k}', name=k)


In [ ]:
valsize = int(len(Xs['input_ids']) * .2)
valsize

51378

### 학습 및 평가 데이터 분리

In [ ]:
# Xs = {}
train_X=[]; val_X=[]
# np.savez_compressed(f'Xs_{k}', name=k)
for k in keys:
    # Xs[k] = np.load(f'Xs_{k}')[k]
    val_X.append(Xs[k][:valsize])
    train_X.append(Xs[k][valsize:])
    
del Xs
gc.collect()

286

In [ ]:
# train_all = pd.read_csv('clf_train_all.csv')

val_y, train_y = train_all.iloc[:valsize, :6].values, train_all.iloc[valsize:, :6].values

## 모델 학습
사용자 정의 TFBertForSequenceClassification 사용:


In [ ]:
from tqdm import tqdm
import tensorflow as tf
from tensorflow.nn import sparse_softmax_cross_entropy_with_logits
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import Loss

In [ ]:
class TFBertForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForSequenceClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        # 모든 가능한 분류를 위해 85를 지정했으나, 필요시 줄일 수 있음
        self.classifier = tf.keras.layers.Dense(85,
                                                kernel_initializer=TruncatedNormal(0.02),
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_token = outputs[1] # 문장 임베딩
        pred = self.classifier(cls_token)

        return pred

### loss 함수 정의

각 분류기별 가중치를 튜닝할 필요도 있음
```python
weights={'age':1, 'sex':.1, 'situation':1, 'illness':.01, 'sentG':1, 'sentL':.1}
```

In [ ]:
class MultiCrossEntropy(Loss):
    def __init__(self, weights=None):
        super().__init__()
        if weights == None:
            self.weights={'age':1, 'sex':1, 'situation':1, 'illness':1, 'sentG':1, 'sentL':1}
        else:
            self.weights = weights

    def call(self, y_true, y_pred):
        age_loss = self.weights['age']*sparse_softmax_cross_entropy_with_logits(y_true[:, 0], y_pred[:, :4])
        sex_loss = self.weights['sex']*sparse_softmax_cross_entropy_with_logits(y_true[:, 1], y_pred[:, 4:6])
        situation_loss = self.weights['situation']*sparse_softmax_cross_entropy_with_logits(y_true[:, 2], y_pred[:, 6:18])
        illness_loss = self.weights['illness']*sparse_softmax_cross_entropy_with_logits(y_true[:, 3], y_pred[:, 18:21])
        sentG_loss = self.weights['sentG']*sparse_softmax_cross_entropy_with_logits(y_true[:, 4], y_pred[:, 21:27])
        sentL_loss = self.weights['sentL']*sparse_softmax_cross_entropy_with_logits(y_true[:, 5], y_pred[:, 27:])
        loss = age_loss + sex_loss + situation_loss + illness_loss + sentG_loss + sentL_loss
        return loss

### TPU 설정

In [ ]:
import os

# TPU 작동을 위한 코드 TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.8.176.26:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.8.176.26:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
with strategy.scope():
    model = TFBertForSequenceClassification("klue/bert-base")
    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
    model.compile(optimizer=optimizer, loss=MultiCrossEntropy())

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'bert.embeddings.position_ids', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
model.fit(train_X, train_y, epochs=10, batch_size=64, 
          validation_data=(val_X, val_y))

Epoch 1/10
3212/3212 [==============================] - 491s 153ms/step - loss: 0.8806 - val_loss: 0.6134
Epoch 2/10
3212/3212 [==============================] - 491s 153ms/step - loss: 0.5179 - val_loss: 0.4829
Epoch 3/10
3212/3212 [==============================] - 490s 153ms/step - loss: 0.3978 - val_loss: 0.3864
Epoch 4/10
3212/3212 [==============================] - 492s 153ms/step - loss: 0.3258 - val_loss: 0.3620
Epoch 5/10
3212/3212 [==============================] - 491s 153ms/step - loss: 0.2768 - val_loss: 0.3142
Epoch 6/10
3212/3212 [==============================] - 491s 153ms/step - loss: 0.2370 - val_loss: 0.2769
Epoch 7/10
3212/3212 [==============================] - 490s 153ms/step - loss: 0.2052 - val_loss: 0.2621
Epoch 8/10
3212/3212 [==============================] - 490s 152ms/step - loss: 0.1841 - val_loss: 0.2365
Epoch 9/10
3212/3212 [==============================] - 490s 153ms/step - loss: 0.1633 - val_loss: 0.2187
Epoch 10/10
3212/3212 [=======================

## 모델 평가 및 예측

### TPU에서의 모델 저장:
1. 학습 모델(model)의 tf.train.Checkpoint로 학습 파라미터 저장
2. 신규 모델(m) 인스턴스 생성
3. 신규 모델(m)의 Checkpoint로 복원



In [ ]:
# not applicable
# model.save('./simple_clf1/')

In [ ]:
# checkpoint = tf.train.Checkpoint(model=model)
# checkpoint_path = './simple_clf1'
# local_device_option = tf.train.CheckpointOptions(experimental_io_device="/job:localhost")
# save_path  = checkpoint.save(checkpoint_path, options=local_device_option)

In [ ]:
# save_path = './simple_clf1'
# save_path

'./simple_clf1'

In [ ]:
!ls

 1.prep_chatbot_data.ipynb   qa1_single-supporting-fact_test_kor.txt
 1_simple_classifier.ipynb   qa1_single-supporting-fact_train_kor.txt
 문장2.xlsx		     simple_clf1-1.index
 문장3.xlsx		     simple_clf1-1_temp
 문장4.xlsx		     simple_clf1.data-00000-of-00001
 감정분류_AI모델	     simple_clf1.index
 all_chat.csv		     test.csv
'gpt_train_data (1).csv'     train_chat_raw.gsheet
'gpt_train_data (2).csv'     training.csv
 gpt_train_data.csv	     training.gsheet
 gpt_val_data.csv	     training.xlsx
 KoBERT			     Untitled0.ipynb
 KoBERT_multi_lable.ipynb    validation.csv
 make_classifier.ipynb	     validation.gsheet
 model.h5		     validation.xlsx
 prep_data.csv


### 저장된 모델을 이용한 예측 수행:

In [ ]:
save_path = './simple_clf1-1'
m = TFBertForSequenceClassification("klue/bert-base")
chkpt = tf.train.Checkpoint(model=m)
local_device_option = tf.train.CheckpointOptions(experimental_io_device="/job:localhost")
chkpt.restore(save_path, options=local_device_option)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'bert.embeddings.position_ids', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
# 1batch의 문장 idx들을 가져와서
val1 = []
for ids in val_X:
    val1.append(ids[:1, :])

val1

[array([[    2,   850,  2052,  1415, 13964,   732,  2116,  3737,  1158,
          2052,   882,  1567,  2073,  5885,  2051,    18,  3667,  4017,
          2170,  9562,   717,    18,     3,  4426,  2052,  4245,  2470,
          3706,  2170,  1158,  2170,  6179,  2415,  2112,  3717,  2052,
          3732,  1378,  1560,  2279,  2067,  2918, 10283,    18,     3,
          4426,  2052,  4245,  6396,   732,  1158,  4159,  2138,  1521,
          1892,  1295,  1513, 16809,  4419,  2811,    18,     3,  3842,
          1889,  2460,  1158,  2052,  4828, 12193,  2651,  6301,    35,
             3,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,

In [ ]:
pred = m.predict(val1)[0]
print(pred.shape)
pred

(85,)


array([ 9.044173  , -3.9320667 , -4.270226  , -3.8680046 , -0.1939852 ,
        0.855103  , -1.4495195 , -0.23412052,  5.3086543 , -2.1918144 ,
       -4.386992  , -1.0170028 , 11.04675   , -5.2132454 , -2.9568388 ,
       -1.0851936 , -1.7383993 , -3.5974557 ,  1.6687621 ,  4.677913  ,
       -7.9104576 , -4.0463586 , -0.6082583 ,  8.672636  , -2.6196518 ,
       -4.667093  , -0.2040013 ,  1.8400655 , -2.8175757 , -2.9001057 ,
       -3.1635745 , -6.4520764 ,  6.744145  , -1.8288387 , -2.611946  ,
       -2.5419872 ,  9.415617  ,  1.5133746 , -1.6767431 , -2.002986  ,
       -1.55392   , -4.325133  , -2.293946  , -2.108801  ,  2.349339  ,
       -2.5991073 , -1.5397404 , -0.31928837,  9.164736  , -2.9414513 ,
        3.1011286 , -4.553733  ,  3.7865324 , -3.1364026 , -0.14450312,
       -3.8475215 , -2.8429093 , -0.8774321 ,  5.1351433 ,  7.5032706 ,
       -1.6756344 , -5.394273  , -2.335246  , -1.6315199 , -2.4063575 ,
       -1.4023051 , -1.9565251 , -2.4623523 ,  5.9519076 , -0.94

### 타겟값과 비교

In [ ]:
def target2list(arr):
  if arr.ndim == 1:
    return [arr[:4], arr[4:6], arr[6:18], arr[18:21], arr[21:27], arr[27:]]
  else:
    return [arr[:,:4], arr[:,4:6], arr[:,6:18], arr[:,18:21], arr[:,21:27], arr[:,27:]]

In [ ]:
[np.argmax(arr) for arr in target2list(pred)]

[0, 1, 6, 1, 2, 9]

In [ ]:
val_y[0]

array([ 0,  1,  6,  1,  2, 21])

### 정확도 계산

In [ ]:
# 각 target별 정확도를 생성합니다.
def evaluate(m, val_X, val_y):
    scores = m.predict(val_X)
    preds = np.vstack([np.argmax(arr, axis=1) for arr in target2list(scores)]).T
    return np.mean(val_y == preds, axis=0)

# accs = evaluate(m, val_X, val_y)
# finalAcc = accs * class_weights / class_weights.sum()